## 1. Importacion De Librerias

Se importan las librerías necesarias para la evaluación de modelos: numpy y pandas para manipulación de datos, scikit-learn para modelos y métricas, joblib para cargar modelos guardados, y os para manejo de rutas de archivos.

In [ ]:
# Librerías para manipulación de datos
import numpy as np
import pandas as pd

# Modelos de clasificación (solo para referencia de tipos)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Manejo de rutas y archivos
import os

# Carga y guardado de modelos
import joblib

# Métricas de evaluación
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score
)

## 2. Definicion De Datos, Modelos y Direcrtorios

Se definen las constantes del proyecto: los 9 tipos de conjuntos de datos a evaluar (original, estandarizado, normalizado y sus variantes con PCA), los 4 modelos de clasificación (KNN, SVM, Naive Bayes, Random Forest), el número de folds para validación cruzada (5), y las rutas a los directorios de datos y modelos.

In [ ]:
# Lista de tipos de conjuntos de datos a evaluar
DATASETS_DIRS = [
    "estandarizado",
    "estandarizado_PCA80",
    "estandarizado_PCA95",
    "normalizado",
    "normalizado_PCA80",
    "normalizado_PCA95",
    "original",
    "original_PCA80",
    "original_PCA95",
]

# Nombres de los modelos a evaluar
MODELS = [
    "KNN",
    "SVM",
    "NaiveBayes",
    "RandomForest"
]

# Número de folds en validación cruzada
K_FOLDS = 5

# Ruta al directorio de datos de prueba
DATA_DIR = os.path.join("..", "data")

if not os.path.exists(DATA_DIR):
    raise FileNotFoundError(f"Data directory '{DATA_DIR}' does not exist. Please ensure the data is available.")

# Ruta al directorio donde están guardados los modelos entrenados
MODELS_DIR = os.path.join("..", "models")

if not os.path.exists(MODELS_DIR):
    raise FileNotFoundError(f"Models directory '{MODELS_DIR}' does not exist. Please ensure you execute the training script first.")

## 3. Carga De Modelos Y Evaluacion

Esta sección carga los modelos entrenados desde disco y los evalúa con los datos de prueba. Para cada combinación de tipo de dato, fold y modelo, se calculan múltiples métricas de rendimiento (accuracy, precision, recall, F1-score, ROC-AUC, sensitivity, specificity, FPR, FNR) usando estrategia macro-averaging. Los resultados se guardan en CSV: uno con predicciones y probabilidades, otro con las métricas calculadas.

### Funciones Auxiliares

**`load_model(data_type, model_name, model_iteration, model_dir)`**  
Deserializa modelos entrenados desde archivos `.pkl` usando joblib. Construye la ruta jerárquica: `models/{data_type}/{model_name}/model_fold_{iteration}.pkl`

**`test_model_and_save_results(X_test, y_test, model)`**  
Función principal de evaluación que:
- Genera predicciones de clase (`predict`) y probabilidades (`predict_proba`)
- Calcula métricas estándar de scikit-learn con `average='macro'` (media aritmética simple entre clases)
- Integra métricas personalizadas de `calculate_multiclass_metrics`
- Retorna dos DataFrames: uno con métricas agregadas y otro con predicciones individuales + probabilidades por clase

**`calculate_multiclass_metrics(y_true, y_pred)`**  
Implementa estrategia **One-vs-Rest (OvR)** para clasificación multiclase:
La Estrategia consiste en ir calculando los FP, FN por cada clase de manera que el resto de clases se juntan en una haciendo del problema multi-class un problema binario.
- Para cada clase $c$, binariza el problema: clase $c$ vs resto
- Calcula matriz de confusión binaria (TP, TN, FP, FN)
- Computa métricas por clase: Sensitivity (TPR), Specificity (TNR), FPR, FNR
- Retorna **macro-averaging**: $\text{metric} = \frac{1}{K}\sum_{i=1}^{K} \text{metric}_i$ donde $K$ es el número de clases

In [ ]:
def load_model(data_type, model_name, model_iteration, model_dir):
    """Carga un modelo entrenado desde disco"""
    model_path = os.path.join(model_dir, f"{data_type}", f"{model_name}", f"model_fold_{model_iteration}.pkl")
    model = joblib.load(model_path)
    return model

def test_model_and_save_results(X_test, y_test, model):
    """Evalúa el modelo y calcula métricas de rendimiento"""
    
    # Predice etiquetas y probabilidades
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    metrics = {}
    
    # Métricas básicas
    metrics['accuracy'] = accuracy_score(y_test, y_pred)
    
    # Métricas con promedio macro (trata todas las clases por igual)
    metrics['precision'] = precision_score(y_test, y_pred, average='macro')
    metrics['recall'] = recall_score(y_test, y_pred, average='macro')
    metrics['f1_score'] = f1_score(y_test, y_pred, average='macro')
    metrics['roc_auc'] = roc_auc_score(y_test, y_pred_proba, average='macro', multi_class='ovr')
    
    # Métricas multiclase personalizadas
    multiclass_metrics = calculate_multiclass_metrics(y_test, y_pred)
    
    metrics.update(multiclass_metrics)
    
    # DataFrame con etiquetas verdaderas, predicciones y probabilidades por clase
    predicted_probability_df = pd.DataFrame(y_pred_proba, columns=[f'Prob_Class_{i}' for i in range(y_pred_proba.shape[1])])
    predicted_probability_df.insert(0, 'True_Label', y_test)
    predicted_probability_df.insert(1, 'Predicted', y_pred)
    
    
    metrics_df = pd.DataFrame([metrics])
    
    return metrics_df, predicted_probability_df
    
    

def calculate_multiclass_metrics(y_true, y_pred):
    """Calcula métricas adicionales para clasificación multiclase usando estrategia One-vs-Rest"""
    
    metrics = {}
    
    unique_classes = np.unique(y_true)
    
    sensitivity_list = []
    specificity_list = []
    fpr_list = []
    fnr_list = []
    
    # Para cada clase, calcula métricas binarias
    for cls in unique_classes:
        y_true_binary = (y_true == cls).astype(int)
        y_pred_binary = (y_pred == cls).astype(int)
        
        # Matriz de confusión
        TP = np.sum((y_true_binary == 1) & (y_pred_binary == 1))
        TN = np.sum((y_true_binary == 0) & (y_pred_binary == 0))
        FP = np.sum((y_true_binary == 0) & (y_pred_binary == 1))
        FN = np.sum((y_true_binary == 1) & (y_pred_binary == 0))
        
        # Cálculo de métricas por clase
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0  # TPR, Recall
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0  # TNR
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 0  # Tasa de falsos positivos
        fnr = FN / (FN + TP) if (FN + TP) > 0 else 0  # Tasa de falsos negativos
        
        sensitivity_list.append(sensitivity)
        specificity_list.append(specificity)
        fpr_list.append(fpr)
        fnr_list.append(fnr)
    
    # Promedio macro de métricas
    metrics['sensitivity'] = np.mean(sensitivity_list)
    metrics['specificity'] = np.mean(specificity_list)
    metrics['fpr'] = np.mean(fpr_list)
    metrics['fnr'] = np.mean(fnr_list)
    
    return metrics
    

# Loop principal de evaluación: itera sobre tipos de datos, folds y modelos
for data_type in DATASETS_DIRS:
    
    data_path = os.path.join(DATA_DIR, data_type)
    
    if not os.path.exists(data_path):
        print(f"Data path '{data_path}' does not exist. Skipping...")
        continue
    
    for fold in range(K_FOLDS):
        
        # Carga datos de prueba del fold actual
        fold_path = os.path.join(data_path, f"test_{fold + 1}_{data_type}.csv")
        
        X_test = pd.read_csv( fold_path ).values[:, :-1]
        y_test = pd.read_csv( fold_path ).values[:, -1]
        
        for model_name in MODELS:
            
            # Carga el modelo entrenado
            model = load_model(data_type, model_name, fold + 1, MODELS_DIR)
            
            print(f"Evaluating Model: {model_name}, Data Type: {data_type}, Fold: {fold + 1}")
            
            # Evalúa y obtiene métricas
            metrics_df, predicted_probability_df = test_model_and_save_results(X_test, y_test, model)
            
            # Prepara directorio de resultados
            results_dir = os.path.join("..", "results", data_type, model_name)
            
            if not os.path.exists(results_dir):
                os.makedirs(results_dir)
            
            # Guarda predicciones y probabilidades
            results_path = os.path.join(results_dir, f"results_predictions_fold_{fold + 1}.csv")
            predicted_probability_df.to_csv(results_path, index=False)
            
            print(f"Results saved at: {results_path}")
            
            # Guarda métricas
            metrics_path = os.path.join(results_dir, f"results_metrics_fold_{fold + 1}.csv")
            metrics_df.to_csv(metrics_path, index=False)
            
            print(f"Metrics saved at: {metrics_path}")


## 4. Ensemble De Modelos Y Evaluacion Final

Se implementan tres técnicas de ensemble para combinar las predicciones de los 4 modelos base:

1. **Votación**: Cada modelo vota por una clase y se elige la más votada (hard voting)
2. **Media de probabilidades**: Se promedian las probabilidades de todos los modelos y se elige la clase con mayor probabilidad promedio (soft voting)
3. **Mediana de probabilidades**: Similar a la media pero usando mediana, más robusto ante valores extremos

Los resultados de cada técnica se guardan en directorios separados (Ensemble_Votacion, Ensemble_Media, Ensemble_Mediana).

In [29]:
# 1. Votacion

for data_type in DATASETS_DIRS:
    
    path = os.path.join("..", "results", data_type)
    
    if not os.path.exists(path):
        print(f"Results path '{path}' does not exist. Skipping...")
        continue
    
    for fold in range(K_FOLDS):
        
        knn_csv = os.path.join(path, "KNN", f"results_predictions_fold_{fold + 1}.csv")
        svm_csv = os.path.join(path, "SVM", f"results_predictions_fold_{fold + 1}.csv")
        nb_csv = os.path.join(path, "NaiveBayes", f"results_predictions_fold_{fold + 1}.csv")
        rf_csv = os.path.join(path, "RandomForest", f"results_predictions_fold_{fold + 1}.csv")
        
        knn_df = pd.read_csv(knn_csv)
        svm_df = pd.read_csv(svm_csv)
        nb_df = pd.read_csv(nb_csv)
        rf_df = pd.read_csv(rf_csv)
        
        true_labels = knn_df['True_Label'].values
        
        predictions = np.array([
            knn_df['Predicted'].values,
            svm_df['Predicted'].values,
            nb_df['Predicted'].values,
            rf_df['Predicted'].values
        ]).astype(int)
        
        final_predictions = []
        
        for i in range(predictions.shape[1]):
            counts = np.bincount(predictions[:, i])
            final_pred = np.argmax(counts)
            final_predictions.append(final_pred)
        
        votacion_df = pd.DataFrame({
            'True_Label': true_labels,
            'Predicted': final_predictions
        })
        
        votacion_csv = os.path.join(path, "Ensemble_Votacion", f"results_predictions_fold_{fold + 1}.csv")
        votacion_dir = os.path.dirname(votacion_csv)
        
        if not os.path.exists(votacion_dir):
            os.makedirs(votacion_dir)
    
        votacion_df.to_csv(votacion_csv, index=False)
        
        
# 2. Por Media De Probabilidades

for data_type in DATASETS_DIRS:
    
    path = os.path.join("..", "results", data_type)
    
    if not os.path.exists(path):
        print(f"Results path '{path}' does not exist. Skipping...")
        continue
    
    for fold in range(K_FOLDS):
        
        knn_csv = os.path.join(path, "KNN", f"results_predictions_fold_{fold + 1}.csv")
        svm_csv = os.path.join(path, "SVM", f"results_predictions_fold_{fold + 1}.csv")
        nb_csv = os.path.join(path, "NaiveBayes", f"results_predictions_fold_{fold + 1}.csv")
        rf_csv = os.path.join(path, "RandomForest", f"results_predictions_fold_{fold + 1}.csv")
        
        knn_df = pd.read_csv(knn_csv)
        svm_df = pd.read_csv(svm_csv)
        nb_df = pd.read_csv(nb_csv)
        rf_df = pd.read_csv(rf_csv)
        
        true_labels = knn_df['True_Label'].values
        
        probas = np.array([
            knn_df.filter(like='Prob_Class_').values,
            svm_df.filter(like='Prob_Class_').values,
            nb_df.filter(like='Prob_Class_').values,
            rf_df.filter(like='Prob_Class_').values
        ])
        
        probas_media = np.mean(probas, axis=0)
        
        final_predictions = np.argmax(probas_media, axis=1)
        
        media_df = pd.DataFrame({
            'True_Label': true_labels,
            'Predicted': final_predictions,
            **{f'Prob_Class_{i}': probas_media[:, i] for i in range(probas_media.shape[1])}
        })
        
        media_csv = os.path.join(path, "Ensemble_Media", f"results_predictions_fold_{fold + 1}.csv")
        media_dir = os.path.dirname(media_csv)
        
        if not os.path.exists(media_dir):
            os.makedirs(media_dir)
    
        media_df.to_csv(media_csv, index=False)
        
        
# 3. Por Mediana De Probabilidades

for data_type in DATASETS_DIRS:
    
    path = os.path.join("..", "results", data_type)
    
    if not os.path.exists(path):
        print(f"Results path '{path}' does not exist. Skipping...")
        continue
    
    for fold in range(K_FOLDS):
        
        knn_csv = os.path.join(path, "KNN", f"results_predictions_fold_{fold + 1}.csv")
        svm_csv = os.path.join(path, "SVM", f"results_predictions_fold_{fold + 1}.csv")
        nb_csv = os.path.join(path, "NaiveBayes", f"results_predictions_fold_{fold + 1}.csv")
        rf_csv = os.path.join(path, "RandomForest", f"results_predictions_fold_{fold + 1}.csv")
        
        knn_df = pd.read_csv(knn_csv)
        svm_df = pd.read_csv(svm_csv)
        nb_df = pd.read_csv(nb_csv)
        rf_df = pd.read_csv(rf_csv)
        
        true_labels = knn_df['True_Label'].values
        
        probas = np.array([
            knn_df.filter(like='Prob_Class_').values,
            svm_df.filter(like='Prob_Class_').values,
            nb_df.filter(like='Prob_Class_').values,
            rf_df.filter(like='Prob_Class_').values
        ])
        
        probas_mediana = np.median(probas, axis=0)
        
        final_predictions = np.argmax(probas_mediana, axis=1)
        
        mediana_df = pd.DataFrame({
            'True_Label': true_labels,
            'Predicted': final_predictions,
            **{f'Prob_Class_{i}': probas_mediana[:, i] for i in range(probas_mediana.shape[1])}
        })
        
        mediana_csv = os.path.join(path, "Ensemble_Mediana", f"results_predictions_fold_{fold + 1}.csv")
        mediana_dir = os.path.dirname(mediana_csv)
        
        if not os.path.exists(mediana_dir):
            os.makedirs(mediana_dir)
    
        mediana_df.to_csv(mediana_csv, index=False)